imports

In [ ]:
# %% Imports -----------------------------------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer

from statsmodels.stats.outliers_influence import variance_inflation_factor

import sys
sys.path.insert(0, "../src")   # keep original relative module path
import SCORE_functions as sf     # project‑specific helpers stay the same

load data:
- data of phase1 will not be considered for NLP for now as we are predicting sco2
- in a later investigation phase it might be of a great value. EX. if we want to look at text and sub-category sco

In [ ]:
# import phase1 data
# phase1_path = "../data/alltextdataphase01.xlsx"
# df1 = pd.read_excel(phase1_path)

In [ ]:
# import phase2 data
phase2_path = "../data/alltextdataphase02.xlsx"
df2 = pd.read_excel(phase2_path)

investigate

In [ ]:
# display df2
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df2.head(4)

In [18]:
# display columns and their indices
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(list(enumerate(df2.columns)))

[(0, 'Timestamp'),
 (1, 'code'),
 (2, 'المنطقة'),
 (3, '7- الخبرة "مجال التدخّل والمجموعات المستهدفة"'),
 (4, 'كيف يمكن وصم عمل الجهة الفاعلة'),
 (5, '8- الوضع القانوني'),
 (6, 'لماذا توقفت الجهة الفاعلة عن العمل؟'),
 (7, '9- الوضع المالي'),
 (8, '10- التغطية الجغرافيّة'),
 (9, '12- رابط قنوات التواصل الاجتماعي'),
 (10, 'التقديم و التوصيف'),
 (11,
  '14- الثقة في الجهات الفاعلية محلياً: هل يمكن للجهة الفاعلة، المساهمة في حل المشاكل المحلية؟'),
 (12,
  'تحليل: إلى أي درجة تثق المجتمعات المحلية في هذه الجهة، و إلى أي درجة تساهم هذه الجهة الفاعلة في بناء الثقة ضمن المجتمع المحلي؟'),
 (13,
  '15- القدرة على الاستجابة: هل الجهة الفاعلة قادرة على دعم المجتمع المحلي في قضايا مثل، تأمين الخدمات وإيصال احتياجات المجتمع المحلي للجهات المعنية بحلها، ومتابعتها؟'),
 (14,
  'تحليل: إلى أي درجة تساهم هذه الجهة في الإستجابة للاحتياجات و المشاكل و إلى أي درجة يلجأ لها المجتمعي المحلي؟'),
 (15, '  التغير في المعيار الأول'),
 (16,
  '16- القرار المجتمعي – التأثير مدى وصول المجتمع للموارد (المشاركة): هل ت

In [22]:
# removing non textual data
keep_idx_list = [1, 6, 10, 11, 13, 15, 16, 19, 21, 23, 24, 26, 28, 29, 31, 34, 35, 37, 39, 40, 42, 44, 45]
rm_idx_list = [0, 2, 3, 4, 5, 7, 8, 9, 12, 14, 17, 18, 20, 22, 25, 27, 30, 32, 33, 36, 38, 41, 43, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]

df2 = df2.drop(df2.columns[rm_idx_list], axis=1)

In [24]:
# renaming columns
mapping = {
    0: "code",
    1: "why stop working",
    2: "inital info",
    3: "Q14",
    4: "Q15",
    5: "change in criterion1",
    6: "Q16",
    7: "Q17",
    8: "Q18",
    9: "change in criterion2",
    10: "Q19",
    11: "Q20",
    12: "change in criterion3",
    13: "Q21",
    14: "Q22",
    15: "change in criterion4",
    16: "Q23",
    17: "Q24",
    18: "change in criterion5",
    19: "Q25",
    20: "Q26",
    21: "change in criterion6",
    22: "the general change?",
}

df2.columns = [mapping[i] for i in range(df2.shape[1])]


In [26]:
# melt df2 (turn into code text pairs)
code_text_df = df2.melt(id_vars="code", value_name="text")
code_text_df = code_text_df[["code", "text"]]

In [32]:
print(df2.isna().sum())

code                      9
why stop working        675
inital info             812
Q14                       9
Q15                       9
change in criterion1      9
Q16                       9
Q17                       9
Q18                       9
change in criterion2      9
Q19                       9
Q20                       9
change in criterion3      9
Q21                       9
Q22                       9
change in criterion4      9
Q23                       9
Q24                       9
change in criterion5      9
Q25                       9
Q26                       9
change in criterion6      9
the general change?      31
dtype: int64


In [33]:
print(code_text_df.isna().sum())

code     198
text    1689
dtype: int64


In [35]:
# remove NANs
code_text_df = code_text_df.dropna()

In [36]:
code_text_df.shape

(16373, 2)

code_text_df is ready we add to it sco2:

In [47]:
# import phase2 data
alldatatoML_path = "../data/alldatatoML.xlsx"
ML_scores = pd.read_excel(alldatatoML_path)

In [ ]:
ML_scores.head(1)

In [49]:
# ML_scores is only Code and all scores2
sco2_cols = [c for c in ML_scores.columns if c.startswith("sco2 - ")]
ML_scores = ML_scores[["Code"] + sco2_cols]

In [ ]:
ML_scores.head(5)

In [51]:
print(ML_scores.isna().sum())

Code                                                                    0
sco2 - 1.1. Trust & engagement in local institutions                    4
sco2 - 1.2. Responsiveness                                             10
sco2 - 2.1. Community decision-making influence                         8
sco2 - 2.2. General consideration about civic life                      0
sco2 - 2.3. Community accessibility to resources (Participation)        5
sco2 - 2.4. Inclusion/exclusion                                         1
sco2 - 3.1. Collective identification                                   2
sco2 - 3.2. Role of key influencers                                    16
sco2 - 4.1. Economic security                                           4
sco2 - 4.2. Community accessibility to resources (Economy)              3
sco2 - 4.3. Perception of social threat                                 8
sco2 - 5.1. Social distance                                             1
sco2 - 5.2. Networking and horizontal 

In [52]:
# fillna in ML_scores
for c in sco2_cols:
    ML_scores.loc[:, c] = ML_scores[c].fillna(ML_scores["sco2 - Actor's relevance"])

In [53]:
ML_scores.shape

(807, 23)

In [63]:
merged = code_text_df.merge(ML_scores, left_on="code", right_on="Code", how="outer")

In [55]:
print(merged.isna().sum())

code                                                                    0
text                                                                    0
Code                                                                   95
sco2 - 1.1. Trust & engagement in local institutions                   95
sco2 - 1.2. Responsiveness                                             95
sco2 - 2.1. Community decision-making influence                        95
sco2 - 2.2. General consideration about civic life                     95
sco2 - 2.3. Community accessibility to resources (Participation)       95
sco2 - 2.4. Inclusion/exclusion                                        95
sco2 - 3.1. Collective identification                                  95
sco2 - 3.2. Role of key influencers                                    95
sco2 - 4.1. Economic security                                          95
sco2 - 4.2. Community accessibility to resources (Economy)             95
sco2 - 4.3. Perception of social threa

In [65]:
merged = merged.drop(columns=["Code"])

In [66]:
merged.shape

(16373, 24)

In [67]:
merged = merged.dropna()

In [68]:
merged.shape

(16278, 24)

In [69]:
print(merged.isna().sum())

code                                                                   0
text                                                                   0
sco2 - 1.1. Trust & engagement in local institutions                   0
sco2 - 1.2. Responsiveness                                             0
sco2 - 2.1. Community decision-making influence                        0
sco2 - 2.2. General consideration about civic life                     0
sco2 - 2.3. Community accessibility to resources (Participation)       0
sco2 - 2.4. Inclusion/exclusion                                        0
sco2 - 3.1. Collective identification                                  0
sco2 - 3.2. Role of key influencers                                    0
sco2 - 4.1. Economic security                                          0
sco2 - 4.2. Community accessibility to resources (Economy)             0
sco2 - 4.3. Perception of social threat                                0
sco2 - 5.1. Social distance                        

In [ ]:
merged.head(2)

now to the translation:

In [ ]:
# export data to translate
output_path_datatoEN = "../data/alltextdatatotranslation.xlsx"
merged.to_excel(output_path_datatoEN, index=False)
print(f"Preprocessing finished – file written to {output_path_datatoEN}")

Preprocessing finished – file written to ../data/alldatatotranslation.xlsx


In [92]:
# import translated data
translateddata_path = "../data/alltextdatatranslated.xlsx"
dataset_translated = pd.read_excel(translateddata_path)

In [ ]:
dataset_translated.tail(2)

In [94]:
# make the final dataframe
dataset_translated.iloc[:, 1] = dataset_translated.iloc[:, -1]
dataset = dataset_translated.drop(dataset_translated.columns[-2:], axis=1)

In [ ]:
# export data to NLTK
output_path_alltextdatatopreprocessing = "../data/alltextdatatopreprocessing.xlsx"
dataset.to_excel(output_path_alltextdatatopreprocessing, index=False)
print(f"file written to {output_path_alltextdatatopreprocessing}")

file written to ../data/alltextdatatoNLTK.xlsx
